In [1]:
import pandas as pd
import numpy as np
import psycopg2 as pg
import geopandas as gpd
import networkx as nx
from tqdm.auto import tqdm

tqdm.pandas()

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
buildings = gpd.read_parquet("/home/gk/jupyter/masterplanning/output_data/buildings.parquet")

In [57]:
class Provision:
    def __init__(self, SUB_GROUP: str = "schools", updated_block_info: dict = None, blocks=None):
        self.SUB_GROUP = SUB_GROUP
        self.updated_block_info = updated_block_info
        self.blocks = blocks
        # self.g = None

    def prepare_graph(self):
        # Тут нужно включить впн
        engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")
        # GROUPS = ('Детский сад', 'Школа', 'Рекреационная зона')
        # SUB_GROUP = GROUPS[0]
        matrix = pd.read_pickle("/home/gk/jupyter/MMG_SPB.pkl")
        blocks_geom = self.blocks.copy()

        buildings = gpd.read_parquet("/home/gk/jupyter/masterplanning/output_data/buildings.parquet")
        res = (
            gpd.sjoin(blocks_geom, buildings, predicate="intersects", how="left")
            .drop(columns=["id_right", "index_right"])
            .groupby("id_left")
            .agg({"population_balanced": "sum", "living_area": "sum"})
        )

        res.reset_index(drop=False, inplace=True)
        res["living_area"] = res["living_area"].apply(lambda x: True if x > 0 else False)
        res.rename(columns={"living_area": "is_living"}, inplace=True)
        # print(res.loc[6630, :])
        blocks_geom = res.merge(blocks_geom, right_on="id", left_on="id_left")
        blocks_geom.drop(columns=["id_left"], inplace=True)
        living_blocks = blocks_geom.loc[:, ["id", "geometry"]].sort_values(by="id").reset_index(drop=True)

        service_blocks_df = gpd.read_postgis(
            f"select capacity, geometry as geom from all_services where city_service_type_code in ('{self.SUB_GROUP}') "
            "and city_id=1",
            con=engine,
        )

        service_blocks_df.rename(columns={"geom": "geometry"}, inplace=True)

        service_blocks_df["geometry"] = service_blocks_df["geometry"].convex_hull
        service_blocks_df["geometry"] = service_blocks_df["geometry"].to_crs(32636)
        service_blocks_df["geometry"] = service_blocks_df["geometry"].centroid

        blocks_geom = gpd.GeoDataFrame(blocks_geom, geometry="geometry", crs=32636)

        service_blocks_df["geometry"] = service_blocks_df["geometry"].set_crs(32636)
        service_blocks_df = service_blocks_df.set_geometry("geometry")

        service_blocks_df = (
            gpd.sjoin(blocks_geom, service_blocks_df, predicate="intersects")
            .groupby("id")
            .agg(
                {
                    "capacity": "sum",
                }
            )
        )
        # print(service_blocks_df)
        if self.updated_block_info:

            print(service_blocks_df.loc[self.updated_block_info["block_id"], "capacity"])
            service_blocks_df.loc[self.updated_block_info["block_id"], "capacity"] += self.updated_block_info[
                "school_capacity"
            ]
            print(service_blocks_df.loc[self.updated_block_info["block_id"], "capacity"])

            blocks_geom.loc[self.updated_block_info["block_id"], "population_balanced"] = self.updated_block_info[
                "population"
            ]

        blocks_geom_dict = blocks_geom[["id", "population_balanced", "is_living"]].set_index("id").to_dict()
        service_blocks_dict = service_blocks_df.to_dict()["capacity"]

        blocks_list = matrix.loc[
            matrix.index.isin(service_blocks_df.index.astype("Int64")), matrix.columns.isin(living_blocks["id"])
        ]

        ACCS_TIME = 30
        self.g = nx.Graph()

        for idx in tqdm(list(blocks_list.index)):

            blocks_list_tmp = blocks_list[blocks_list.index == idx]
            blocks_list.columns = blocks_list.columns.astype(int)
            blocks_list_tmp = blocks_list_tmp[blocks_list_tmp < ACCS_TIME].dropna(axis=1)
            blocks_list_tmp_dict = blocks_list_tmp.transpose().to_dict()[idx]

            for key in blocks_list_tmp_dict.keys():

                if key != idx:
                    self.g.add_edge(idx, key, weight=round(blocks_list_tmp_dict[key], 1))

                else:

                    self.g.add_node(idx)

                self.g.nodes[key]["population"] = blocks_geom_dict["population_balanced"][int(key)]
                self.g.nodes[key]["is_living"] = blocks_geom_dict["is_living"][int(key)]

                if key != idx:
                    try:
                        if self.g.nodes[key][f"is_{self.SUB_GROUP}_service"] != 1:
                            self.g.nodes[key][f"is_{self.SUB_GROUP}_service"] = 0
                            self.g.nodes[key][f"provision_{self.SUB_GROUP}"] = 0
                            self.g.nodes[key][f"id_{self.SUB_GROUP}"] = 0
                    except KeyError:
                        self.g.nodes[key][f"is_{self.SUB_GROUP}_service"] = 0
                        self.g.nodes[key][f"provision_{self.SUB_GROUP}"] = 0
                        self.g.nodes[key][f"id_{self.SUB_GROUP}"] = 0
                else:
                    self.g.nodes[key][f"is_{self.SUB_GROUP}_service"] = 1
                    self.g.nodes[key][f"{self.SUB_GROUP}_capacity"] = service_blocks_dict[key]
                    self.g.nodes[key][f"provision_{self.SUB_GROUP}"] = 0
                    self.g.nodes[key][f"id_{self.SUB_GROUP}"] = 0

                if self.g.nodes[key]["is_living"] == True:
                    self.g.nodes[key][f"population_prov_{self.SUB_GROUP}"] = 0
                    self.g.nodes[key][f"population_unprov_{self.SUB_GROUP}"] = 0

    def get_stats(self):
        g = self.g
        if not g:
            return 0

        blocks_service = 0
        blocks_bad = 0
        blocks_living = 0
        total = 0
        for key in g:
            total += 1
            try:
                if g.nodes[key]["is_living"] == True:
                    blocks_living += 1
                elif g.nodes[key][f"is_{self.SUB_GROUP}_service"] == 1:
                    blocks_service += 1
            except KeyError as ex:
                blocks_bad += 1

        print(f"количество кварталов c сервисом {self.SUB_GROUP}: {blocks_service}")
        print(f"количество жилых кварталов: {blocks_living}")
        print(f"количество кварталов всего: {total}")
        print(f"количество кварталов c ошибкой: {blocks_bad}")

    def get_provision(self):
        SUB_GROUP = self.SUB_GROUP
        g = self.g

        for node in g.nodes:
            if g.nodes[node][f"is_{SUB_GROUP}_service"] == 1:
                capacity = g.nodes[node][f"{SUB_GROUP}_capacity"]
                neighbors = list(g.neighbors(node))
                if g.nodes[node]["is_living"] == True and g.nodes[node]["population"] > 0:
                    if g.nodes[node][f"population_prov_{self.SUB_GROUP}"] == 0:
                        population = g.nodes[node]["population"]
                    else:
                        population = g.nodes[node][f"population_unprov_{self.SUB_GROUP}"]

                    if SUB_GROUP == "schools":
                        load = (population / 1000) * 120
                    elif SUB_GROUP == "kindergartens":
                        load = (population / 1000) * 60
                    elif SUB_GROUP == "recreational_areas":
                        load = population

                    if g.nodes[node][f"provision_{SUB_GROUP}"] < 100 and load <= capacity:
                        prov_proc = (population * 100) / population
                        g.nodes[node][f"provision_{SUB_GROUP}"] += prov_proc
                        capacity -= load
                        g.nodes[node][f"id_{SUB_GROUP}"] = node
                        g.nodes[node][f"population_prov_{SUB_GROUP}"] += population
                        g.nodes[node][f"population_unprov_{SUB_GROUP}"] = 0

                    elif g.nodes[node][f"provision_{self.SUB_GROUP}"] < 100 and load > capacity:
                        if capacity > 0:
                            if self.SUB_GROUP == "schools":
                                prov_people = (capacity * 1000) / 120
                            elif self.SUB_GROUP == "kindergartens":
                                prov_people = (capacity * 1000) / 60
                            elif self.SUB_GROUP == "recreational_areas":
                                prov_people = capacity
                            unprov_people = population - prov_people
                            prov_pop = (prov_people * 100) / population
                            capacity -= capacity
                            g.nodes[node][f"population_prov_{self.SUB_GROUP}"] += prov_people
                            g.nodes[node][f"population_unprov_{self.SUB_GROUP}"] += unprov_people
                            g.nodes[node][f"id_{self.SUB_GROUP}"] = node
                            g.nodes[node][f"provision_{self.SUB_GROUP}"] += prov_pop

                neighbors = list(g.neighbors(node))

                for neighbor in neighbors:
                    if g.nodes[neighbor]["is_living"] == True and g.nodes[neighbor]["population"] > 0:
                        if capacity > 0:
                            # population = g.nodes[neighbor]['population']
                            if g.nodes[neighbor][f"population_prov_{self.SUB_GROUP}"] == 0:
                                population = g.nodes[neighbor]["population"]
                            else:
                                population = g.nodes[neighbor][f"population_unprov_{self.SUB_GROUP}"]

                            if self.SUB_GROUP == "schools":
                                load = (population / 1000) * 120
                            elif self.SUB_GROUP == "kindergartens":
                                load = (population / 1000) * 60
                            elif self.SUB_GROUP == "recreational_areas":
                                load = population
                            if g.nodes[neighbor][f"provision_{self.SUB_GROUP}"] < 100 and load <= capacity:
                                prov_proc = (population * 100) / population
                                g.nodes[neighbor][f"provision_{self.SUB_GROUP}"] += prov_proc
                                capacity -= load
                                g.nodes[neighbor][f"id_{self.SUB_GROUP}"] = node
                                g.nodes[neighbor][f"population_prov_{self.SUB_GROUP}"] += population
                                # g.nodes[neighbor][f'population_unprov_{self.SUB_GROUP}'] = 0
                            elif g.nodes[neighbor][f"provision_{self.SUB_GROUP}"] < 100 and load > capacity:

                                if capacity > 0:
                                    if self.SUB_GROUP == "schools":
                                        prov_people = (capacity * 1000) / 120
                                    elif self.SUB_GROUP == "kindergartens":
                                        prov_people = (capacity * 1000) / 60
                                    elif self.SUB_GROUP == "recreational_areas":
                                        prov_people = capacity

                                    unprov_people = population - prov_people
                                    prov_pop = (prov_people * 100) / population
                                    capacity -= capacity
                                    g.nodes[neighbor][f"population_prov_{self.SUB_GROUP}"] += prov_people
                                    g.nodes[neighbor][f"population_unprov_{self.SUB_GROUP}"] += unprov_people
                                    g.nodes[neighbor][f"id_{self.SUB_GROUP}"] = node
                                    g.nodes[neighbor][f"provision_{self.SUB_GROUP}"] += prov_pop
                            else:
                                g.nodes[neighbor][f"provision_{self.SUB_GROUP}"] += 0
        self.g = g

    def get_geo(self):
        g = self.g
        blocks = self.blocks.copy()
        blocks[f"provision_{self.SUB_GROUP}"] = 0
        blocks[f"id_{self.SUB_GROUP}"] = 0
        blocks[f"population_prov_{self.SUB_GROUP}"] = 0
        blocks[f"population_unprov_{self.SUB_GROUP}"] = 0
        blocks[f"provision_{self.SUB_GROUP}"] = 0

        # print(blocks)

        for n in g:
            # print(n, end="\r")
            indx = blocks[blocks.index == n].index[0]
            if g.nodes[n]["is_living"] == True:
                if g.nodes[n].get(f"id_{self.SUB_GROUP}") is not None:
                    blocks.loc[indx, f"id_{self.SUB_GROUP}"] = g.nodes[n][f"id_{self.SUB_GROUP}"]
                    blocks.loc[indx, f"population_prov_{self.SUB_GROUP}"] = g.nodes[n][
                        f"population_prov_{self.SUB_GROUP}"
                    ]
                    blocks.loc[indx, f"population_unprov_{self.SUB_GROUP}"] = g.nodes[n][
                        f"population_unprov_{self.SUB_GROUP}"
                    ]
                    blocks.loc[indx, f"provision_{self.SUB_GROUP}"] = g.nodes[n][f"provision_{self.SUB_GROUP}"]
                else:
                    blocks[f"population_unprov_{self.SUB_GROUP}"][indx] = g.nodes[n][
                        f"population_unprov_{self.SUB_GROUP}"
                    ]

        blocks[f"id_{self.SUB_GROUP}"] = blocks[f"id_{self.SUB_GROUP}"].astype(int)
        blocks[f"population_prov_{self.SUB_GROUP}"] = blocks[f"population_prov_{self.SUB_GROUP}"].astype(int)
        blocks[f"population_unprov_{self.SUB_GROUP}"] = blocks[f"population_unprov_{self.SUB_GROUP}"].astype(int)
        blocks[f"provision_{self.SUB_GROUP}"] = blocks[f"provision_{self.SUB_GROUP}"].astype(int)

        return blocks

In [4]:
import pickle

with open("../output_data/balanced_result_block.pkl", "rb") as f:
    updated_block = pickle.load(f)

In [5]:
updated_block

{'area': 269.91357042657916,
 'population': 49547.0,
 'b': 24.000000000000004,
 'green_coef_G': 9.0,
 'living_area': 63.8661511,
 'school_area': 4.7,
 'school_capacity': 2500,
 'kindergarten_area': 4.4,
 'kindergarten_capacity': 1120,
 'green_area': 43.0201,
 'G_min_capacity': 71700.16666666667,
 'G_max_capacity': 35850.083333333336,
 'green_coef_G_capacity': 47800.11111111111,
 'op_area': 0.061914000000000004,
 'parking1_area': 15.602328000000002,
 'parking2_area': 15.16893,
 'block_id': 2157}

In [6]:
# b1 = gpd.read_parquet('../output_data/BLOCKS_AND_BUILDINGS_INFO.parquet')
blocks = gpd.read_file("/home/gk/jupyter/masterplanning/output_data/blocks_CUTTED.geojson")

In [8]:
blocks.drop(columns=["id"], inplace=True)
blocks.reset_index(drop=False, inplace=True)
blocks.rename(columns={"index": "id"}, inplace=True)

blocks

,id,geometry
0,0,"POLYGON ((316983.653 6644038.696, 316986.485 6..."
1,1,"POLYGON ((315287.843 6646411.375, 315288.850 6..."
2,2,"POLYGON ((315582.670 6646809.216, 315582.581 6..."
3,3,"POLYGON ((315264.235 6646596.731, 315274.102 6..."
4,4,"POLYGON ((313479.271 6646628.746, 313479.599 6..."
...,...,...
6189,6189,"POLYGON ((364951.367 6632403.641, 365082.578 6..."
6190,6190,"POLYGON ((365055.659 6632100.991, 364837.693 6..."
6191,6191,"POLYGON ((364591.035 6633403.929, 364597.995 6..."
6192,6192,"POLYGON ((364504.029 6632935.898, 364504.962 6..."


In [58]:
import os

# GROUPS = ('kindergartens', 'schools', 'recreational_areas')
# blocks = gpd.read_parquet('/home/gk/jupyter/masterplanning/output_data/blocks.parquet')

for sub_group in ["schools"]:
    # for sub_group in GROUPS:

    # if not os.path.exists(f'../output_data/{sub_group}.parquet'):
    model = Provision(SUB_GROUP=sub_group, blocks=blocks)
    model.prepare_graph()
    model.get_stats()
    model.get_provision()
    gdf = model.get_geo()
    # gdf.to_parquet(f'../output_data/{sub_group}.parquet')

/usr/local/lib/python3.8/dist-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
100%|██████████| 605/605 [00:10<00:00, 57.03it/s]


количество кварталов c сервисом schools: 51
количество жилых кварталов: 2110
количество кварталов всего: 5522
количество кварталов c ошибкой: 0


In [59]:
""" 
ДО
"""

gdf.loc[2157, :]

id                                                                        2157
geometry                     POLYGON ((340408.2947395761 6639979.697077261,...
provision_schools                                                          100
id_schools                                                                2157
population_prov_schools                                                  28909
population_unprov_schools                                                    0
Name: 2157, dtype: object

In [60]:
import os

# GROUPS = ('kindergartens', 'schools', 'recreational_areas')
# blocks = gpd.read_parquet('/home/gk/jupyter/masterplanning/output_data/blocks.parquet')

# for sub_group in GROUPS:
for sub_group in ["schools"]:

    # if not os.path.exists(f'../output_data/{sub_group}.parquet'):
    model = Provision(SUB_GROUP=sub_group, updated_block_info=updated_block, blocks=blocks)
    model.prepare_graph()
    model.get_stats()
    model.get_provision()
    gdf = model.get_geo()
    # gdf.to_parquet(f'../output_data/{sub_group}.parquet')

/usr/local/lib/python3.8/dist-packages/geopandas/io/sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


3972
6472


100%|██████████| 605/605 [00:10<00:00, 58.26it/s]


количество кварталов c сервисом schools: 51
количество жилых кварталов: 2110
количество кварталов всего: 5522
количество кварталов c ошибкой: 0


In [61]:
""" 
ПОСЛЕ
"""

gdf.loc[2157, :]

id                                                                        2157
geometry                     POLYGON ((340408.2947395761 6639979.697077261,...
provision_schools                                                          100
id_schools                                                                2157
population_prov_schools                                                  49547
population_unprov_schools                                                    0
Name: 2157, dtype: object

In [23]:
updated_block

{'area': 269.91357042657916,
 'population': 49547.0,
 'b': 24.000000000000004,
 'green_coef_G': 9.0,
 'living_area': 63.8661511,
 'school_area': 4.7,
 'school_capacity': 2500,
 'kindergarten_area': 4.4,
 'kindergarten_capacity': 1120,
 'green_area': 43.0201,
 'G_min_capacity': 71700.16666666667,
 'G_max_capacity': 35850.083333333336,
 'green_coef_G_capacity': 47800.11111111111,
 'op_area': 0.061914000000000004,
 'parking1_area': 15.602328000000002,
 'parking2_area': 15.16893,
 'block_id': 2157}

In [62]:
len(gdf.loc[gdf[f"provision_schools"] == 100])

1697

In [63]:
gdf.loc[gdf[f"population_unprov_schools"] > 0]

,id,geometry,provision_schools,id_schools,population_prov_schools,population_unprov_schools
92,92,"POLYGON ((306717.568 6678136.612, 306717.770 6...",46,107,199,233
124,124,"POLYGON ((314894.482 6679507.958, 314894.744 6...",143,358,1824,1030
216,216,"POLYGON ((319793.421 6645990.836, 319793.385 6...",122,207,1142,879
541,541,"POLYGON ((319421.537 6646694.444, 319421.600 6...",178,213,1113,236
571,571,"POLYGON ((320624.168 6654912.212, 320624.085 6...",85,327,682,112
...,...,...,...,...,...,...
5925,5925,"POLYGON ((364022.684 6633357.696, 364022.750 6...",93,5926,23641,214699
6004,6004,"POLYGON ((365734.620 6623835.369, 365726.015 6...",198,5820,7202,74
6013,6013,"POLYGON ((365716.437 6624359.046, 365704.944 6...",141,5829,20657,22881
6028,6028,"POLYGON ((365775.767 6625236.228, 365788.628 6...",175,6005,3914,953
